In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN, Dropout
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix
import itertools
import urllib.request
import requests
import collections
from nltk import FreqDist
from collections import Counter
import seaborn as sns
import requests
import warnings

warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 100)
%config InlineBackend.figure_format='retina'

In [ ]:

'''
CLEAN TWEETS
'''

train_dir = '../input/tweet-sentiment-extraction/train.csv'
tweets_train = pd.read_csv(train_dir, encoding='utf_8')

test_dir = '../input/tweet-sentiment-extraction/test.csv'
tweets_test = pd.read_csv(test_dir, encoding='utf_8')

print(tweets_train.shape)
tweets_train[['text', 'sentiment']].head()

In [ ]:
def plot_word_length(text):
    words = text.split(' ')
    word_lengths = [len(word) for word in words]
    sns.boxplot(word_lengths)
    pass

plt.figure(figsize=(12, 5))
plot_word_length(document)

In [ ]:
# clean text
def clean_tweet(tweet):
    
    # make str
    tweet = str(tweet)
    
    # lower case
    tweet = tweet.lower()
    
    # remove numbers
    #tweet = re.sub(r"\d+", " ", tweet)
    
    # remove urls
    url_pattern = r"https?://\S+|www\.\S+"
    tweet = re.sub(url_pattern, ' ', tweet)
    
    # remove html
    html = r'<.*?>'
    tweet = re.sub(html, ' ', tweet)
    
    # remove puncuations and numbers
    tweet = re.sub(r"""[^\s\w\d'"]+""", ' ', tweet)
    
    # expand contractions
    tweet = expand_contractions(tweet)
    
    # remove extra whitespaces
    tweet = re.sub('\s+', ' ', tweet)
    tweet = tweet.strip()
    
    # tokenize
    words = word_tokenize(tweet)
    
    # remove whitespaces
    words = [word.strip() for word in words]
    
    # remove words shorter than n characters in length
    n = 1
    words = [word for word in words if len(word) > n]
    
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # join words
    tweet = ' '.join(words)
    
    return tweet

def expand_contractions(text):
    contractions = { 
    "ain't": "am not",
    "aint": "am not",
    "aren't": "are not",
    "arent": "are not",
    "can't": "cannot",
        "cant": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldnt": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "didnt": "did not",
    "doesn't": "does not",
    "doesnt": "does not",
    "don't": "do not",
    "dont": "do not",
    "hadn't": "had not",
    "hadnt": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "hasnt": "has not",
    "haven't": "have not",
    "havent": "have not",
    "he'd": "he would",
    "hed": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "Id": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "Im": "I am",
    "I've": "I have",
    "Ive": "I have",
    "isn't": "is not",
    "isnt": "is not",
    "it'd": "it would",
    "itd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "lets": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "wont": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    "gonna": "going to",
    "wanna": "want to",
    "'t": "not",
    "'s": "is",
    "'ll": "will",
    "'d": "would",
    "'re": "are",
    "'ve": "have",
    "w/": "with",
    "w/o": "without",
    "&quot;": "'",
    "&amp;": "and",
    "&lt;": "less than"
    }
    
    # change dictionary to lower case
    contractions = dict((k.lower(), v.lower()) for k,v in contractions.items())

    # tokenize words
    words = re.split(r"\s+|\.", text.lower())
    
    # expand contractions
    new_words = []
    for word in words:
        if(word in contractions.keys()):
            new_words.append(contractions[word])
            #print(word, "changed to", contractions[word])
            pass
        else:
            new_words.append(word)
            pass
        pass
    text = ' '.join(new_words)
    
    return text

In [ ]:
tweets_train['clean_tweet'] = tweets_train.text.apply(lambda tweet: clean_tweet(tweet))
tweets_test['clean_tweet'] = tweets_test.text.apply(lambda tweet: clean_tweet(tweet))

# split data into train and validation
X = tweets_train.drop('sentiment', axis=1).values
y = tweets_train.sentiment.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=4)

# concatenate X and y for train and validation sets
def make_dataframe(X, y, X_names, y_name):
    '''
    Create a pandas dataframe using features (X) and target (y) variable.
    Input:
        X: A 2D array of features
        y: the target variable
        X_names: names of features in the order that they are present in X
        y_name: name of the target variable
    
    Output:
        df: pandas dataframe object
    '''
    X = pd.DataFrame(X, columns=X_names)
    y = pd.Series(y, name=y_name)
    df = pd.concat([X, y], axis=1)
    return df

X_names = ['textID', 'text', 'selected_text', 'clean_tweet']
y_name = 'sentiment'
tweets_train = make_dataframe(X_train, y_train, X_names=X_names, y_name=y_name)
tweets_validation = make_dataframe(X_val, y_val, X_names=X_names, y_name=y_name)

In [ ]:
'''
ANALYZE TWEETS
'''

# distribution of sentiment
tweets_train.sentiment.value_counts(normalize=True)

# create large corpus of all tweets
document = []
for tweet in tweets_train['clean_tweet']:
    document.append(str(tweet))
    pass

document = ' '.join(document)

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(y=tweets_train.sentiment)
plt.ylabel('Sentiment')
plt.xlabel('Number of tweets')
plt.title('Distribution of Sentiment')

# target class distribution
print(tweets_train.sentiment.value_counts()/tweets_train.shape[0])

In [ ]:

# plots
plt.figure(figsize=(12, 7))
def plot_word_frequency(words, top_n=10):
    word_freq = FreqDist(words)                # or word_freq = Counter(text)
    words = [element[0] for element in word_freq.most_common(top_n)]
    frequencies = [element[1] for element in word_freq.most_common(top_n)]
    plot = sns.barplot(words, frequencies)
    plt.title('Most Common Words in Tweets')
    plt.ylabel('Frequency')
    return plot

plot_word_frequency(word_tokenize(document), top_n=15)

In [ ]:
def num_words(sentence):
    words = sentence.split(' ')
    return len(words)

lengths = tweets_train.text.apply(lambda tweet: num_words(str(tweet)))

plt.figure(figsize=(15, 7))
sns.boxplot(lengths, tweets_train.sentiment)
plt.xlabel('Number of words in tweet.')
plt.ylabel('Sentiment')
plt.title('Tweet Lengths')

In [ ]:
'''
VADER
'''

!pip install vaderSentiment

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import operator
from sklearn import metrics

# predict label using VADER
def sentiment_labeler(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)
    score.pop('compound')
    label = max(score.items(), key=operator.itemgetter(1))[0]
    return expand_label(label)

def expand_label(label):
    expansion = {'neu': 'neutral', 'pos': 'positive', 'neg': 'negative'}
    return expansion[label]

tweets_train['vader_sentiment'] = tweets_train.text.apply(lambda text: sentiment_labeler(str(text)))
tweets_test['vader_sentiment'] = tweets_test.text.apply(lambda text: sentiment_labeler(str(text)))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, normalize=False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    classes = np.unique(y_true)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        precision=4
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], precision)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    pass

# confusion matrix on test set
y_true = tweets_test.sentiment.values
y_pred = tweets_test.vader_sentiment.values

plt.figure(figsize=(12, 7))
plot_confusion_matrix(y_true, y_pred, normalize=True)

print("Accuracy = {:.4f}".format(metrics.accuracy_score(y_true, y_pred)))

In [ ]:

'''
LSTM
'''

# segregate tweets and sentiment
#X_train = tweets_train.text.astype(str)
X_train = tweets_train.clean_tweet.astype(str)
y_train = tweets_train.sentiment.astype(str)
X_train[:5]
y_train[:5]

#X_val = tweets_validation.text.astype(str)
X_val = tweets_validation.clean_tweet.astype(str)
y_val = tweets_validation.sentiment.astype(str)

#X_test = tweets_test.text.astype(str)
X_test = tweets_test.clean_tweet.astype(str)
y_test = tweets_test.sentiment.astype(str)

In [ ]:
# encode words in tweets to integers
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)
X_train_encoded = word_tokenizer.texts_to_sequences(X_train)
X_val_encoded = word_tokenizer.texts_to_sequences(X_val)
X_test_encoded = word_tokenizer.texts_to_sequences(X_test)
X_train_encoded[:5]

# look at lengths tweets
lengths = [len(seq) for seq in X_train_encoded]
print("Length of longest sentence: {}".format(max(lengths)))
plt.boxplot(lengths)

In [ ]:
def sentiment2int(array, reverse=False):
    dictionary = {'positive': 0, 'neutral': 1, 'negative': 2}
    if reverse:
        reverse_dictionary = {v: k for k, v in dictionary.items()}
        int2sentiment = [reverse_dictionary[item] for item in array]
        return int2sentiment
    else:
        sentiment2int = [dictionary[item] for item in array]
        return sentiment2int
    pass

y_train_encoded = sentiment2int(y_train)
y_val_encoded = sentiment2int(y_val)
y_test_encoded = sentiment2int(y_test)
y_train_encoded[:20]

In [ ]:
# pad tweets
MAX_SEQ_LENGTH = 30  # tweets greater than 100 in length will be truncated
X_train_padded = pad_sequences(X_train_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
X_val_padded = pad_sequences(X_val_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
X_test_padded = pad_sequences(X_test_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")

print(X_train_padded[0], "\n"*3)

In [ ]:
# download word2vec embeddings
os.makedirs('../data/')
os.chdir('../data/')
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
!gunzip GoogleNews-vectors-negative300.bin
os.chdir('../working')

In [ ]:
# load word2vec embeddings
path = '../data/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

EMBEDDING_SIZE  = 300
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass
    pass

# check embedding dimension
print("Embeddings shape: {}".format(embedding_weights.shape))

In [ ]:
# one-hot encode sentiments
y_train_one_hot = to_categorical(y_train_encoded)
y_val_one_hot = to_categorical(y_val_encoded)
y_test_one_hot = to_categorical(y_test_encoded)

y_train_one_hot[:10]

In [ ]:
# split data
X_train = X_train_padded
X_val = X_val_padded
X_test = X_test_padded

y_train = y_train_one_hot
y_val = y_val_one_hot
y_test = y_test_one_hot

print("-"*50)
print("TRAINING DATA")
print("-"*50)
print('Shape of input sequences: {}'.format(X_train.shape))
print('Shape of output sequences: {}'.format(y_train.shape))

print('\n')
print("-"*50)
print("VALIDATION DATA")
print("-"*50)
print('Shape of input sequences: {}'.format(X_val.shape))
print('Shape of output sequences: {}'.format(y_val.shape))

print('\n')
print("-"*50)
print("TEST DATA")
print("-"*50)
print('Shape of input sequences: {}'.format(X_test.shape))
print('Shape of output sequences: {}'.format(y_test.shape))

In [ ]:
import keras
from keras import regularizers

# instantiate LSTM model
NUM_CLASSES = y_train.shape[1]
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim     = VOCABULARY_SIZE,         # vocabulary size - number of unique words in data
                         output_dim    = EMBEDDING_SIZE,          # length of vector with which each word is represented
                         input_length  = MAX_SEQ_LENGTH,          # length of input sequence
                         weights       = [embedding_weights],     # word embedding matrix
                         trainable     = False                     # True - update embeddings_weight matrix
))

lstm_model.add(LSTM(8, return_sequences=True))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(LSTM(8, return_sequences=True))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(LSTM(8, return_sequences=False))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(Dense(NUM_CLASSES, activation='softmax'))

# compile
lstm_model.compile(loss      =  'categorical_crossentropy',
                   optimizer =  'adam',
                   metrics   =  ['acc'])

# model summary
lstm_model.summary()

In [ ]:
# fit model on train set and validate on validation set
batch_size= 64
epochs = 20
class_weights = {0: 0.32, 1: 0.4, 2: 0.28}
lstm_training = lstm_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), class_weight=class_weights, verbose=1)

In [ ]:
# accuracy vs epochs
plt.figure(figsize=(15, 8))
plt.plot(lstm_training.history['acc'])
plt.plot(lstm_training.history['val_acc'])
plt.title('LSTM Training')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc="lower right")
plt.show()

# loss vs epochs
plt.figure(figsize=(15, 8))
plt.plot(lstm_training.history['loss'])
plt.plot(lstm_training.history['val_loss'])
plt.title('LSTM Training')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc="lower right")
plt.show()

In [ ]:
# train model on (train + validation) set
X = np.concatenate((X_train, X_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0)

# instantiate LSTM model
NUM_CLASSES = y_train.shape[1]
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim     = VOCABULARY_SIZE,         # vocabulary size - number of unique words in data
                         output_dim    = EMBEDDING_SIZE,          # length of vector with which each word is represented
                         input_length  = MAX_SEQ_LENGTH,          # length of input sequence
                         weights       = [embedding_weights],     # word embedding matrix
                         trainable     = False                    # True - update embeddings_weight matrix
))

lstm_model.add(LSTM(8, return_sequences=True))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(LSTM(8, return_sequences=True))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(LSTM(8, return_sequences=False))
lstm_model.add(BatchNormalization())
#lstm_model.add(Dropout(0.2))

lstm_model.add(Dense(NUM_CLASSES, activation='softmax'))

# compile
lstm_model.compile(loss      =  'categorical_crossentropy',
                   optimizer =  'adam',
                   metrics   =  ['acc'])

# model summary
lstm_model.summary()

# fit model
batch_size= 64
epochs = 20
class_weights = {0: 0.32, 1: 0.4, 2: 0.28}
lstm_training = lstm_model.fit(X, y, batch_size=batch_size, epochs=epochs, class_weight=class_weights, verbose=1)

In [ ]:
# make predictions on test set
y_pred = lstm_model.predict_classes(X_test)
y_pred = sentiment2int(y_pred, reverse=True)

In [ ]:
y_test = np.argmax(y_test, axis = 1)
y_test = sentiment2int(y_test, reverse=True)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, normalize=False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    classes = np.unique(y_true)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        precision=4
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], precision)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    pass

In [ ]:
plt.figure(figsize=(12, 9))
plot_confusion_matrix(y_test, y_pred, normalize=True)

print("LSTM accuracy = {:.4f}".format(metrics.accuracy_score(y_test, y_pred)))